# Story:

I don't comment much for kernels but for this kernel I will.

First of all I am very thankful to: @akensert, @ajinomoto132 and @adityaecdrid who helped me a lot in finding the mistake I was making. It was due to them that I was able to find the mistake in my training code.

@akensert shared a kernel with data-processing similar to mine but a different model and loss function.
This kernel was written using Tensorflow. You can checkout the kernel here: https://www.kaggle.com/akensert/complete-tf2-1-mixed-precision-implementation

Please upvote @akensert's kernel mentioned above! :)

Thus, my plan began to replicate the same score in pytorch. Previously I was using BCEWithLogitsLoss. The TF kernel used Cross Entropy loss. This was one of the major differences. Another major difference was using the last two hidden states instead of just the last one.

So, I tried and failed.

I then tried again, cleaned up my code, started comparing line-by-line and failed again.

After 2 days of frustration, I made a discussion post asking for help: https://www.kaggle.com/c/tweet-sentiment-extraction/discussion/141019

Then came @ajinomoto132. He mentioned he had replicated the model and gladly shared his code to help me out!!! His code can be found here: https://www.kaggle.com/ajinomoto132/starter-kernel-in-pytorch .

Please upvote @ajinomoto132's kernel mentioned above! :)

After a few more hours of struggle, I was able to find the mistake I was doing. It was a stupid mistake of not using `.from_pretrained` when using BertModel. Quite stupid I would say.

Since the community helped me so much, I am releasing the fixed version of my code which is also much cleaner than the previous versions of my code.

I love this community! Thank you for all the help!

ストーリー。
カーネルについてはあまりコメントしないのですが、このカーネルについてはコメントします。

まず第一に、私はとても感謝しています。akensertさん、@ajinomoto132さん、@adityaecdridさんに感謝しています。彼らのおかげで、私のトレーニングコードの間違いを見つけることができました。

akensertさんは、私と似たようなデータ処理をするカーネルを共有してくれましたが、モデルと損失関数は異なります。このカーネルはTensorflowを使って書かれています。カーネルはここでチェックアウトできます: https://www.kaggle.com/akensert/complete-tf2-1-mixed-precision-implementation

上記の@akensertさんのカーネルをupvoteしてください! :)

このように、私の計画は、pytorchで同じスコアを複製するために始めました。以前はBCEWithLogitsLossを使っていた。TFカーネルではCross Entropy lossを使っていた。これが大きな違いの一つだった。もう一つの大きな違いは、最後の２つの隠れた状態を使うのではなく、最後の２つの隠れた状態を使うことでした。

そこで、私は試してみましたが失敗しました。

その後、もう一度やってみて、コードをきれいにして、行ごとに比較を始めて、また失敗しました。

2 日間のフラストレーションの後、私は助けを求めるディスカッション・ポストを作成しました: https://www.kaggle.com/c/tweet-sentiment-extraction/discussion/141019

そこへ@ajinomoto132さんがやってきました。彼はモデルを複製したと言って、喜んで自分のコードをシェアしてくれました！彼のコードはここにあります。彼のコードはここにあります: https://www.kaggle.com/ajinomoto132/starter-kernel-in-pytorch .

上で述べた @ajinomoto132 さんのカーネルをアップバッドしてください! :)

さらに数時間の格闘の後、私は自分がやっていた間違いを見つけることができました。それは、BertModel を使うときに .from_pretrained を使わないという愚かなミスでした。非常に愚かなミスと言えるでしょう。

コミュニティに助けられたので、私のコードの修正版をリリースします。

私はこのコミュニティが大好きです。助けてくれてありがとうございました

www.DeepL.com/Translator（無料版）で翻訳しました。

# All the important imports

In [1]:
import os
import torch
import pandas as pd
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch.optim import lr_scheduler

from sklearn import model_selection
from sklearn import metrics
import transformers
import tokenizers
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm.autonotebook import tqdm
from util import utils # /home/tidal/python/Tweet_Sentiment_Extraction/work/util/utils.py

from pytorch_memlab import profile



In [2]:
class config:
    MAX_LEN = 192
    TRAIN_BATCH_SIZE = 8
    VALID_BATCH_SIZE = 16
    EPOCHS = 20
    #BERT_PATH = "/home/tidal/ML_Data/Tweet_Sentiment_Extraction/_env_bert-base-uncased-using-pytorch/bert-base-uncased/"
    MODEL_PATH = "/home/tidal/ML_Data/Tweet_Sentiment_Extraction/Output_Model_bin/model/"
    TRAINING_FILE = "/home/tidal/ML_Data/Tweet_Sentiment_Extraction/_env_bert-base-uncased-using-pytorch/tweet_train_folds/train_folds.csv"
    #ver3:NLPmodelをBERTからRobeltaに換える。
    ROBERTA_PATH = "/home/tidal/ML_Data/Tweet_Sentiment_Extraction/_env_convRobelta_/roberta-base"
    TOKENIZER = tokenizers.ByteLevelBPETokenizer(
    vocab_file=f"{ROBERTA_PATH}/vocab.json", 
    merges_file=f"{ROBERTA_PATH}/merges.txt", 
    lowercase=True,
    add_prefix_space=True
    )

# Data Processing(train,test)

In [3]:
# memo
tok_tweet = config.TOKENIZER.encode("positive and negative and neutral")
input_ids_orig = tok_tweet.ids[1:-1]
tweet_offsets = tok_tweet.offsets[1:-1]
print(input_ids_orig) # 単語ごとのvocab-list(英語辞典)のインデックス(番号(you->2017,pen->7279等))
print(tweet_offsets) # 文章内での単語の位置(spaceも含まれてる？)


print('    positive and negative and neutral       '.strip())
print('    positive and negative and neutral       ')
print("positive and negative and neutral\n".strip('a'))
print("positive and negative and neutral\n".strip().split())
print('www.example.com'.strip('cmowz.') )

[8, 2430, 8]
[(9, 13), (13, 22), (22, 26)]
positive and negative and neutral
    positive and negative and neutral       
positive and negative and neutral

['positive', 'and', 'negative', 'and', 'neutral']
example


In [4]:
#memo
print([5])
print([100])
print([100]+[5])
print([1]*5)
sentiment_id = {
     'positive': 1313,
     'negative': 2430,
     'neutral': 7974
}
print([sentiment_id["positive"]])
print([sentiment_id["neutral"]])

[5]
[100]
[100, 5]
[1, 1, 1, 1, 1]
[1313]
[7974]


In [5]:
#memo

In [6]:
# 参考：https://www.kaggle.com/akensert/complete-tf2-1-mixed-precision-implementation
# 最終的にtorch.utils.data.DataLoaderへ突っ込む。
def process_data(tweet, selected_text, sentiment, tokenizer, max_len):
    tweet = " " + " ".join(str(tweet).split())
    selected_text = " " + " ".join(str(selected_text).split())

    len_st = len(selected_text) - 1
    idx0 = None
    idx1 = None

    for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
        if " " + tweet[ind: ind+len_st] == selected_text:
            idx0 = ind
            idx1 = ind + len_st - 1
            break

    char_targets = [0] * len(tweet)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1
    
    tok_tweet = tokenizer.encode(tweet)
    input_ids_orig = tok_tweet.ids
    tweet_offsets = tok_tweet.offsets
    
    target_idx = []
    for j, (offset1, offset2) in enumerate(tweet_offsets):
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)
    
    targets_start = target_idx[0]
    targets_end = target_idx[-1]

    sentiment_id = {
        'positive': 1313,
        'negative': 2430,
        'neutral': 7974
    }
    
    #使うNLPモデルに応じて形式が違うっぽい。
    input_ids = [0] + [sentiment_id[sentiment]] + [2] + [2] + input_ids_orig + [2]
    token_type_ids = [0, 0, 0, 0] + [0] * (len(input_ids_orig) + 1)
    mask = [1] * len(token_type_ids)
    tweet_offsets = [(0, 0)] * 4 + tweet_offsets + [(0, 0)]
    targets_start += 4
    targets_end += 4

    padding_length = max_len - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + ([1] * padding_length)
        mask = mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)
        tweet_offsets = tweet_offsets + ([(0, 0)] * padding_length)
    
    # ○：修正の期待大、△：修正の期待微妙、☓：修正の必要なし
    return {
        'ids': input_ids, # ○：[101,(positive/negative/neutralに対応するインデックス),102,(textのトークン(インデックス)),102]
        'mask': mask, # ☓：マスク
        'token_type_ids': token_type_ids, # ☓：前列のsentiment以外の部分のマスク
        'targets_start': targets_start, # ☓：selected_textの始まりの要素番号
        'targets_end': targets_end, # ☓：selected_textの終わりの要素番号
        'orig_tweet': tweet, # △：生のtext
        'orig_selected': selected_text, # △：生のselected_text
        'sentiment': sentiment, # ○：生のsentiment
        'offsets': tweet_offsets # ☓：#文章内での単語の位置(spaceも含まれてる？)
    }

# Data loader

In [7]:
# memo
import datetime
class Test(object):
    def __getitem__(self, x):
        the_hour = datetime.datetime.now().hour
        if x == "greeting":
            if 0 <= the_hour < 6:
                return "Don't to speak to me"
            elif 6 <= the_hour < 10:
                return "Goodmorning"
            elif 10 <= the_hour < 13:
                return "Goodnoon"
            elif 13 <= the_hour < 17:
                return "Goodafternoon"
            else:
                return "Goodevening"
        elif x == "sit":
            return "down"
        elif x == "stand":
            return "up"
        else:
            return "What?"
test = Test()
print("greeting: "+test["greeting"])
print("sit: "+test["sit"])
print("stand: "+test["stand"])
print(" : "+test[""])

greeting: Goodafternoon
sit: down
stand: up
 : What?


In [8]:
class TweetDataset:
    def __init__(self, tweet, sentiment, selected_text):
        self.tweet = tweet
        self.sentiment = sentiment
        self.selected_text = selected_text
        self.tokenizer = config.TOKENIZER
        self.max_len = config.MAX_LEN
    
    def __len__(self):
        return len(self.tweet)

    def __getitem__(self, item):
        data = process_data(
            self.tweet[item], 
            self.selected_text[item], 
            self.sentiment[item],
            self.tokenizer,
            self.max_len
        )
        # process_dataのreturn値をtorch.tensor化させただけ。
        return {
            'ids': torch.tensor(data["ids"], dtype=torch.long),
            'mask': torch.tensor(data["mask"], dtype=torch.long),
            'token_type_ids': torch.tensor(data["token_type_ids"], dtype=torch.long),
            'targets_start': torch.tensor(data["targets_start"], dtype=torch.long),
            'targets_end': torch.tensor(data["targets_end"], dtype=torch.long),
            'orig_tweet': data["orig_tweet"],
            'orig_selected': data["orig_selected"],
            'sentiment': data["sentiment"],
            'offsets': torch.tensor(data["offsets"], dtype=torch.long)
        }

# The Model

In [9]:
# memo
w = torch.empty(3, 5)
nn.init.normal_(w)

tensor([[-1.7148, -0.3180,  0.1963, -0.1679,  1.4556],
        [-0.4571, -1.5386, -0.5511,  0.0904,  1.1705],
        [ 3.0277, -0.4441,  1.2359,  0.0040, -1.9921]])

In [10]:
#memo
x = torch.randn(2, 3)
print(x)
print(torch.cat((x, x, x), dim = 0))
print(torch.cat((x, x, x), dim = 1))
print(torch.cat((x, x, x), dim = -1))

tensor([[ 0.9855,  1.2791,  0.2467],
        [-1.0155,  0.0599,  0.8850]])
tensor([[ 0.9855,  1.2791,  0.2467],
        [-1.0155,  0.0599,  0.8850],
        [ 0.9855,  1.2791,  0.2467],
        [-1.0155,  0.0599,  0.8850],
        [ 0.9855,  1.2791,  0.2467],
        [-1.0155,  0.0599,  0.8850]])
tensor([[ 0.9855,  1.2791,  0.2467,  0.9855,  1.2791,  0.2467,  0.9855,  1.2791,
          0.2467],
        [-1.0155,  0.0599,  0.8850, -1.0155,  0.0599,  0.8850, -1.0155,  0.0599,
          0.8850]])
tensor([[ 0.9855,  1.2791,  0.2467,  0.9855,  1.2791,  0.2467,  0.9855,  1.2791,
          0.2467],
        [-1.0155,  0.0599,  0.8850, -1.0155,  0.0599,  0.8850, -1.0155,  0.0599,
          0.8850]])


In [11]:
# transformers.PreTrainedModel(config, *inputs, **kwargs)
# https://huggingface.co/transformers/main_classes/model.html?highlight=pretrainedmodel#transformers.PreTrainedModel
# configの内容
# https://huggingface.co/transformers/main_classes/configuration.html#transformers.PretrainedConfig
class TweetModel(transformers.BertPreTrainedModel): # 初期値（コンフィグ）設定用のモデル
    #-------初期化部-----------
    def __init__(self, conf):
        super(TweetModel, self).__init__(conf)
        # ローカルに格納してあるモデル(bert-base-uncased)
        self.roberta = transformers.RobertaModel.from_pretrained(config.ROBERTA_PATH, config=conf) #ver3:NLPmodelをBERTからRobeltaに換える。
        #self.bert = transformers.BertModel.from_pretrained(config.BERT_PATH, config=conf)
        # 0.1の確率でモデルの要素の一部をドロップアウト。
        self.drop_out = nn.Dropout(0.1) #ver5
        # input:768*2, Output:2
        self.l0 = nn.Linear(768 * 2, 2)
        torch.nn.init.normal_(self.l0.weight, std=0.02) # self.l0の重みを初期化。
    
    #-------構成部-----------
    # nn.module(つまりこのモデルクラスの__init__)が初期化されるたびに最初に呼び出される。
    def forward(self, ids, mask, token_type_ids):
        #bert(転移学習モデル)層
        #output:転移学習モデルのoutput+embedding(input)のタプル
        #_, _, out = self.bert(
        #ver3:NLPmodelをBERTからRobeltaに換える。
        _, _, out = self.roberta(
            ids,            # ○：[101,(positive/negative/neutralに対応するインデックス),102,(textのトークン(インデックス)),102]
            attention_mask=mask,            # token_type_idsの要素数分の[1]ベクトル（マスク)
            token_type_ids=token_type_ids   # [0,0,0,(textの単語数+1だけ要素1を付与)]
        )
        
        # bertモデルのoutputとinput([101,(positive/negative/neutralに対応するインデックス),102,(textのトークン(インデックス)),102])
        # を結合する。
        out = torch.cat((out[-1], out[-2]), dim=-1)
        #ドロップアウト
        out = self.drop_out(out)
        #出力層。
        # logit:logistic回帰の逆関数、オッズ比
        logits = self.l0(out)

        #targetと同様、selected_textの始め・終わりの位置を出力する。
        start_logits, end_logits = logits.split(1, dim=-1)

        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)

        #それぞれ1×レコード数の横ベクトルを返す。
        return start_logits, end_logits

In [12]:
#memo
print("実験体")
a = torch.arange(8.)
print(a)
a = torch.reshape(a, (4, 2))
print(a)
print("\n")

print("~sprit~")
start_logits, end_logits = a.split(1, dim=-1)
print(start_logits)
print(end_logits)
print(start_logits.size())
print(end_logits.size())
print("\n")

print("~squeeze~")
start_logits = start_logits.squeeze(-1)
end_logits = end_logits.squeeze(-1)
print(start_logits)
print(end_logits)
print(start_logits.size())
print(end_logits.size())
print("\n")

実験体
tensor([0., 1., 2., 3., 4., 5., 6., 7.])
tensor([[0., 1.],
        [2., 3.],
        [4., 5.],
        [6., 7.]])


~sprit~
tensor([[0.],
        [2.],
        [4.],
        [6.]])
tensor([[1.],
        [3.],
        [5.],
        [7.]])
torch.Size([4, 1])
torch.Size([4, 1])


~squeeze~
tensor([0., 2., 4., 6.])
tensor([1., 3., 5., 7.])
torch.Size([4])
torch.Size([4])




# Loss Function

In [13]:
#ver5
def pos_weight(pred_tensor, pos_tensor, neg_weight=1, pos_weight=1):
    # neg_weight for when pred position < target position
    # pos_weight for when pred position > target position
    #→±があるなら打ち消してしまう可能性あるのでは？
    #　→絶対値で重みをつけるやり方に変えてみる　→　☓：ひどく低い結果(0.4)
    gap = torch.argmax(pred_tensor, dim=1) - pos_tensor
    gap = gap.type(torch.float32)
    return torch.where(gap < 0, -neg_weight * gap, pos_weight * gap)

In [14]:
def loss_fn(start_logits, end_logits, start_positions, end_positions):
    # 交差エントロピー関数。以下参考：
    # 公式：https://pytorch.org/docs/stable/nn.html?highlight=crossentropyloss#torch.nn.CrossEntropyLoss
    # 一例：https://qiita.com/Haaamaaaaa/items/58ba0962a8fcd1eeb82e
    loss_fct = nn.CrossEntropyLoss(reduction='none') # do reduction later(reductionとはlossの結果の平均を取ること)　ver5
    # 予測として、select_textのはじめと終わりの位置を予想している。(飛び石的に点在することは想定していない。)
    # はじめと終わりの位置のlossをそれぞれ計算し、それらの和を最終的なlossとして採用。
    start_loss = loss_fct(start_logits, start_positions) * pos_weight(start_logits, start_positions, 1, 1) #ver5
    end_loss = loss_fct(end_logits, end_positions) * pos_weight(end_logits, end_positions, 1, 1) #ver5
    
    #reduction
    start_loss = torch.mean(start_loss) #ver5
    end_loss = torch.mean(end_loss) #ver5
    
    total_loss = (start_loss + end_loss)
    return total_loss

# Training Function

In [15]:
def train_fn(data_loader, model, optimizer, device, scheduler=None):
    model.train() #pytorch特有のやつ（trainモード開始）
    #それぞれ平均として保持する。
    losses = utils.AverageMeter()
    jaccards = utils.AverageMeter()

    tk0 = tqdm(data_loader, total=len(data_loader))
    
    #test用
    test_bi = 0
    # dataのepoch数分回す。（fitting）
    for bi, d in enumerate(tk0):

        #batchサイズ分のデータ
        ids = d["ids"]
        token_type_ids = d["token_type_ids"]
        mask = d["mask"]
        targets_start = d["targets_start"]
        targets_end = d["targets_end"]
        sentiment = d["sentiment"]
        orig_selected = d["orig_selected"]
        orig_tweet = d["orig_tweet"]
        targets_start = d["targets_start"]
        targets_end = d["targets_end"]
        offsets = d["offsets"]

        #device(GPU)用のデータ形式に変換。
        ids = ids.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets_start = targets_start.to(device, dtype=torch.long)
        targets_end = targets_end.to(device, dtype=torch.long)
        
        #学習パラメータの勾配を0に初期化。
        model.zero_grad()
        
        #tester
        #print(ids)
        #print(ids.shape)
        #print(mask)
        #print(mask.shape)
        #print(token_type_ids)
        #print(token_type_ids.shape)
        
        #予測値を出力。
        #予測値はMAX_LEN文字数分のone-hot化された配列として出力される。
        #（つまりレコード数×MAX_LENの配列）
        outputs_start, outputs_end = model(
            ids=ids,
            mask=mask,
            token_type_ids=token_type_ids,
        )
        
        #損失関数を計算。
        loss = loss_fn(outputs_start, outputs_end, targets_start, targets_end)
        #損失関数の勾配を計算。(学習パラメータに足し合わせる)
        loss.backward()
        #最適化実行部
        optimizer.step() 
        #スケジューラ読み込み。(learning lateの大きさ設定。)
        scheduler.step()

        #batchごとにsoftmaxで計算して使用。
        #outputs_start、outputs_endの一つ一つの要素はは0<x<1となる。
        outputs_start = torch.softmax(outputs_start, dim=1).cpu().detach().numpy()
        outputs_end = torch.softmax(outputs_end, dim=1).cpu().detach().numpy()
        jaccard_scores = []
        #tweet(レコード)ごとに処理。
        for px, tweet in enumerate(orig_tweet):
            selected_tweet = orig_selected[px]
            tweet_sentiment = sentiment[px]
            #jaccard scoreを計算
            jaccard_score, _ = calculate_jaccard_score(
                original_tweet=tweet,
                target_string=selected_tweet,
                sentiment_val=tweet_sentiment,
                idx_start=np.argmax(outputs_start[px, :]), #one-hot化配列の最も高い要素(最もそれらしい文字位置)を算出。
                idx_end=np.argmax(outputs_end[px, :]), #one-hot化配列の最も高い要素(最もそれらしい文字位置)を算出。
                offsets=offsets[px]
            )
            jaccard_scores.append(jaccard_score)

        #1epoch分の評価、損失関数の更新。(平均として保存。)
        jaccards.update(np.mean(jaccard_scores), ids.size(0))
        losses.update(loss.item(), ids.size(0))
        #プログレスバーの隣に引数の標準出力を表示。
        tk0.set_postfix(loss=losses.avg, jaccard=jaccards.avg)
        #test用
        test_bi = test_bi + 1
    
    print(f"[Training Data] Jaccard Score = {jaccards.avg}")
    
    #print("tester:bi")
    #print(test_bi)
    #print("tester:outputs_start.shape")
    #print(outputs_start.shape)
    #print("tester:orig_tweet.shape")
    #print(len(orig_tweet))

In [16]:
#dim=0
print("dim=0")
m = nn.Softmax(dim=0)
inp = torch.arange(6.)
inp = torch.reshape(inp, (1, 6))
output = m(inp)
print(inp)
print(output)
print("\n")
#dim=1
print("dim=1")
m = nn.Softmax(dim=1)
inp = torch.arange(6.)
inp = torch.reshape(inp, (1, 6))
output = m(inp)
print(inp)
print(output)
print("\n")
#dim=1
print("dim=1,使用関数")
output = torch.softmax(inp, dim=1).cpu().detach().numpy()
print(inp)
print(output)
out_arg=np.argmax(output[0, :])
print(out_arg)


dim=0
tensor([[0., 1., 2., 3., 4., 5.]])
tensor([[1., 1., 1., 1., 1., 1.]])


dim=1
tensor([[0., 1., 2., 3., 4., 5.]])
tensor([[0.0043, 0.0116, 0.0315, 0.0858, 0.2331, 0.6337]])


dim=1,使用関数
tensor([[0., 1., 2., 3., 4., 5.]])
[[0.00426978 0.01160646 0.03154963 0.08576079 0.233122   0.6336913 ]]
5


# Evaluation Functions

In [17]:
def calculate_jaccard_score(
    original_tweet, 
    target_string, 
    sentiment_val, 
    idx_start, 
    idx_end, 
    offsets,
    verbose=False):
    
    #整合性チェック
    #予測値の終わり位置が始まり位置より小さかった場合、始まりの位置に合わせる。
    if idx_end < idx_start:
        idx_end = idx_start
    
    filtered_output  = ""
    for ix in range(idx_start, idx_end + 1):
        #予測された生の単語(+スペース)を取り出す。
        filtered_output += original_tweet[offsets[ix][0]: offsets[ix][1]]
        #整合性チェック(おかしい場合はスペースのみ出力。)
        if (ix+1) < len(offsets) and offsets[ix][1] < offsets[ix+1][0]:
            filtered_output += " "

    #生の文章が1単語のみの場合そのまま出力。
    #sentimentが"neutralの場合そのまま出力。
    #  →discussionにてneutralの場合ほぼselected_text=textだった話から？
    if sentiment_val == "neutral" or len(original_tweet.split()) < 4:#ver2:2->4
        filtered_output = original_tweet

    #strip：両端の空白を削除。(間のスペースはそのまま。)
    #ターゲットと予測のjaccard score(今回の評価基準)を計算。
    jac = utils.jaccard(target_string.strip(), filtered_output.strip())
    return jac, filtered_output


def eval_fn(data_loader, model, device):
    #評価モード
    model.eval()
    #それぞれ平均として保持する。
    losses = utils.AverageMeter()
    jaccards = utils.AverageMeter()
    
    #勾配計算を無効にする。(無駄なメモリ使用を削減する。)
    #Tensor.backward()を使わない場合の推論時に使う。
    with torch.no_grad():
        tk0 = tqdm(data_loader, total=len(data_loader))
        #batchサイズ分のデータ
        for bi, d in enumerate(tk0):
            ids = d["ids"]
            token_type_ids = d["token_type_ids"]
            mask = d["mask"]
            sentiment = d["sentiment"]
            orig_selected = d["orig_selected"]
            orig_tweet = d["orig_tweet"]
            targets_start = d["targets_start"]
            targets_end = d["targets_end"]
            offsets = d["offsets"].numpy()

            #device(GPU)用のデータ形式に変換。
            ids = ids.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets_start = targets_start.to(device, dtype=torch.long)
            targets_end = targets_end.to(device, dtype=torch.long)

            #予測値を出力。
            #予測値はMAX_LEN文字数分のone-hot化された配列として出力される。
            #（つまりレコード数×MAX_LENの配列）
            outputs_start, outputs_end = model(
                ids=ids,
                mask=mask,
                token_type_ids=token_type_ids
            )
            #損失関数を計算。
            loss = loss_fn(outputs_start, outputs_end, targets_start, targets_end)
            #batchごとにsoftmaxで計算して使用。
            #outputs_start、outputs_endの一つ一つの要素はは0<x<1となる。
            outputs_start = torch.softmax(outputs_start, dim=1).cpu().detach().numpy()
            outputs_end = torch.softmax(outputs_end, dim=1).cpu().detach().numpy()
            jaccard_scores = []
            #tweet(レコード)ごとに処理。
            for px, tweet in enumerate(orig_tweet):
                selected_tweet = orig_selected[px]
                tweet_sentiment = sentiment[px]
                #jaccard scoreを計算
                jaccard_score, _ = calculate_jaccard_score(
                    original_tweet=tweet,
                    target_string=selected_tweet,
                    sentiment_val=tweet_sentiment,
                    idx_start=np.argmax(outputs_start[px, :]),#one-hot化配列の最も高い要素(最もそれらしい文字位置)を算出。
                    idx_end=np.argmax(outputs_end[px, :]), #one-hot化配列の最も高い要素(最もそれらしい文字位置)を算出。
                    offsets=offsets[px]
                )
                jaccard_scores.append(jaccard_score)

            #1epoch分の評価、損失関数の更新。(平均として保存。)
            jaccards.update(np.mean(jaccard_scores), ids.size(0))
            losses.update(loss.item(), ids.size(0))
            #プログレスバーの隣に引数の標準出力を表示。
            tk0.set_postfix(loss=losses.avg, jaccard=jaccards.avg)
    
    print(f"Jaccard = {jaccards.avg}")
    return jaccards.avg

# Training

In [18]:
def run(fold):
    dfx = pd.read_csv(config.TRAINING_FILE)
    
    #traindataは5つに分けている。
    df_train = dfx[dfx.kfold != fold].reset_index(drop=True)
    df_valid = dfx[dfx.kfold == fold].reset_index(drop=True)
    
    #traindataを前処理。
    train_dataset = TweetDataset(
        tweet=df_train.text.values,
        sentiment=df_train.sentiment.values,
        selected_text=df_train.selected_text.values
    )

    # pytorchの書式に合ったデータセットに整形する。
    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=config.TRAIN_BATCH_SIZE,
        num_workers=4 #動かすサブプロセス数
    )

    #validdataを前処理。
    valid_dataset = TweetDataset(
        tweet=df_valid.text.values,
        sentiment=df_valid.sentiment.values,
        selected_text=df_valid.selected_text.values
    )

    # pytorchの書式に合ったデータセットに整形する。
    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=config.VALID_BATCH_SIZE,
        num_workers=2 #動かすサブプロセス数
    )

    #bertモデルの準備。->Robeltaモデルの準備
    device = torch.device("cuda")
    model_config = transformers.RobertaConfig.from_pretrained(config.ROBERTA_PATH)#ver3
    model_config.output_hidden_states = True
    model = TweetModel(conf=model_config)
    model.to(device)

    #学習はnum_train_steps*Batch数*epoch数でtrainデータを全て見るようにできている？
    num_train_steps = int(len(df_train) / config.TRAIN_BATCH_SIZE * config.EPOCHS)
    
    #optimizerに関するパラメータの生成.
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
    ]
    optimizer = AdamW(optimizer_parameters, lr=3e-5)
    
    #learning lateをepochごとに変化させていく。
    #以下参考(公式)
    #https://huggingface.co/transformers/main_classes/optimizer_schedules.html?highlight=get_linear_schedule_with_warmup#transformers.get_linear_schedule_with_warmup
    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=0, 
        num_training_steps=num_train_steps
    )
    
    es = utils.EarlyStopping(patience=2, mode="max")
    print(f"Training is Starting for fold={fold}")
    
    for epoch in range(config.EPOCHS):
        print(f"{epoch + 1}th execute.")
        train_fn(train_data_loader, model, optimizer, device, scheduler=scheduler)
        jaccard = eval_fn(valid_data_loader, model, device)
        print(f"Jaccard Score = {jaccard}")
        #batchの学習と評価を行うたびにチェック(EarlyStopping)を入れる。
        es(jaccard, model, model_path=f"{config.MODEL_PATH}model_{fold}.bin")
        if es.early_stop:
            print("Early stopping")
            break

# work memo
__ver0__<br>
baseline:<br>
・実行回数：4回<br>
・最高点：0.6897446742916279<br>
__ver2__<br>
original_text採用の字数範囲1->3:<br>
・実行回数：3回<br>
・最高点：0.6916938347678977<br>
__ver4__<br>
dropout0.1->0.25:<br>
・実行回数：5回<br>
・最高点：0.6966387911439028<br>
__ver5__<br>
loss_fcの修正(±)&dropout0.25->0.1:<br>
・実行回数：6回<br>
・最高点：0.6886750972565311<br>
__ver5②__<br>
loss_fcの修正(絶対値):<br>
・実行回数：4回<br>
・最高点：0.4435915371559025<br>

# tester memo
TRAIN_BATCH_SIZE = 8<br>
tester:bi<br>
2749<br>
tester:outputs_start.shape<br>
(4, 128)<br>
tester:orig_tweet.shape<br>
4<br>

In [19]:
%%time
run(fold=0)

Training is Starting for fold=0
1th execute.



[Training Data] Jaccard Score = 0.6321614105863652



Jaccard = 0.6649554368022813
Jaccard Score = 0.6649554368022813
Validation score improved (-inf --> 0.6649554368022813). Saving model!
2th execute.



[Training Data] Jaccard Score = 0.6623580003717262



Jaccard = 0.6659654996428275
Jaccard Score = 0.6659654996428275
Validation score improved (0.6649554368022813 --> 0.6659654996428275). Saving model!
3th execute.


...


Jaccard = 0.6886750972565311
Jaccard Score = 0.6886750972565311
Validation score improved (0.6816056767882744 --> 0.6886750972565311). Saving model!
5th execute.



[Training Data] Jaccard Score = 0.6802990004723503



Jaccard = 0.6772304529507653
Jaccard Score = 0.6772304529507653
EarlyStopping counter: 1 out of 2
6th execute.



[Training Data] Jaccard Score = 0.6891377490602278



Jaccard = 0.6642394923278911
Jaccard Score = 0.6642394923278911
EarlyStopping counter: 2 out of 2
Early stopping
CPU times: user 53min 48s, sys: 3min 57s, total: 57min 46s
Wall time: 57min 43s


In [19]:
%%time
run(fold=1)

Training is Starting for fold=1
1th execute.



Jaccard = 0.6794389588079408
Jaccard Score = 0.6794389588079408
Validation score improved (-inf --> 0.6794389588079408). Saving model!
2th execute.



Jaccard = 0.6856715329011135
Jaccard Score = 0.6856715329011135
Validation score improved (0.6794389588079408 --> 0.6856715329011135). Saving model!
3th execute.



Jaccard = 0.6791347160787086
Jaccard Score = 0.6791347160787086
EarlyStopping counter: 1 out of 2
4th execute.



Jaccard = 0.6734254770999576
Jaccard Score = 0.6734254770999576
EarlyStopping counter: 2 out of 2
Early stopping
CPU times: user 26min 21s, sys: 1min 1s, total: 27min 23s
Wall time: 27min 22s


In [20]:
%%time
run(fold=2)

Training is Starting for fold=2
1th execute.



Jaccard = 0.6943632817447951
Jaccard Score = 0.6943632817447951
Validation score improved (-inf --> 0.6943632817447951). Saving model!
2th execute.



Jaccard = 0.6948874824360924
Jaccard Score = 0.6948874824360924
EarlyStopping counter: 1 out of 2
3th execute.



Jaccard = 0.6872569544056131
Jaccard Score = 0.6872569544056131
EarlyStopping counter: 2 out of 2
Early stopping
CPU times: user 20min 34s, sys: 47 s, total: 21min 21s
Wall time: 21min 20s


In [21]:
%%time
run(fold=3)

Training is Starting for fold=3
1th execute.



Jaccard = 0.6928683875678339
Jaccard Score = 0.6928683875678339
Validation score improved (-inf --> 0.6928683875678339). Saving model!
2th execute.



Jaccard = 0.6921212739420334
Jaccard Score = 0.6921212739420334
EarlyStopping counter: 1 out of 2
3th execute.



Jaccard = 0.6824131911143663
Jaccard Score = 0.6824131911143663
EarlyStopping counter: 2 out of 2
Early stopping
CPU times: user 20min 36s, sys: 47.4 s, total: 21min 24s
Wall time: 21min 24s


In [22]:
%%time
run(fold=4)

Training is Starting for fold=4
1th execute.



Jaccard = 0.6995940521388123
Jaccard Score = 0.6995940521388123
Validation score improved (-inf --> 0.6995940521388123). Saving model!
2th execute.



Jaccard = 0.6974747338729177
Jaccard Score = 0.6974747338729177
EarlyStopping counter: 1 out of 2
3th execute.



Jaccard = 0.6911721404604438
Jaccard Score = 0.6911721404604438
EarlyStopping counter: 2 out of 2
Early stopping
CPU times: user 20min 35s, sys: 46.4 s, total: 21min 21s
Wall time: 21min 21s
